Part 1

In [380]:
import numpy as np

In [381]:
with open('input.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    n = lines.index('')
    grid = np.array([list(row) for row in lines[:n]], dtype=str)
    directions = ''.join(lines[n+1:])

In [382]:
robot_position = next(((i, j) for i, row in enumerate(grid) 
                      for j, cell in enumerate(row) if cell == '@'))

direction_moves = {'>': (0, 1), '<': (0, -1), '^': (-1, 0), 'v': (1, 0)}

for direction in directions:
    dr, dc = direction_moves[direction]
    next_pos = (robot_position[0] + dr, robot_position[1] + dc)
    
    if grid[next_pos[0], next_pos[1]] == '.':
        grid[robot_position[0], robot_position[1]], grid[next_pos[0], next_pos[1]] = '.', '@'
        robot_position = next_pos
    elif grid[next_pos[0], next_pos[1]] == 'O':
        n_box = 1
        box_pos = next_pos
        
        while True:
            next_box = (box_pos[0] + dr, box_pos[1] + dc)
            if grid[next_box[0], next_box[1]] == '#':
                break
            elif grid[next_box[0], next_box[1]] == 'O':
                box_pos = next_box
                n_box += 1
                continue
            elif grid[next_box[0], next_box[1]] == '.':
                if direction in '<>':
                    r, c = robot_position
                    slice_range = slice(c, c+(n_box+1)+1) if direction=='>' else slice(c-(n_box+1), c+1)
                    grid[r, slice_range] = np.roll(grid[r, slice_range], 1 if direction=='>' else -1)
                else:
                    r, c = robot_position
                    cells = [grid[r, c]] + [grid[r+dr*(k+1), c] for k in range(n_box+1)]
                    new_cells = np.roll(cells, 1)
                    grid[r, c] = new_cells[0]
                    for k in range(n_box+1):
                        grid[r+dr*(k+1), c] = new_cells[k+1]
                robot_position = (robot_position[0] + dr, robot_position[1] + dc)
            break

p1 = sum(100*i + j for i, row in enumerate(grid) 
         for j, cell in enumerate(row) if cell == 'O')
print(f'Part 1: {p1}')

Part 1: 1511865


Part 2

In [383]:
with open('input.txt') as f:
    lines = [line.strip() for line in f.readlines()]
    n = lines.index('')
    grid = np.array([list(row) for row in lines[:n]], dtype=str)
    directions = ''.join(lines[n+1:])

def transform_grid(grid):
    # Convert single characters to pairs according to rules
    transforms = {'#': '#'*2, 'O': '[]', '.': '.'*2, '@': '@.'}
    return np.array([[c for pair in [transforms[cell] for cell in row] for c in pair] for row in grid])

grid = transform_grid(grid)
robot_position = next(((i, j) for i, row in enumerate(grid) 
                      for j, cell in enumerate(row) if cell == '@'))

def print_grid(grid):
    for row in grid:
        print(''.join(row))

In [384]:
robot_position = next(((i, j) for i, row in enumerate(grid) 
                      for j, cell in enumerate(row) if cell == '@'))

direction_moves = {'>': (0, 1), '<': (0, -1), '^': (-1, 0), 'v': (1, 0)}

for direction in directions:
    dr, dc = direction_moves[direction]
    next_pos = (robot_position[0] + dr, robot_position[1] + dc)
    
    # Next point is empty - just move the robot.
    if grid[next_pos[0], next_pos[1]] == '.':
        grid[robot_position[0], robot_position[1]], grid[next_pos[0], next_pos[1]] = '.', '@'
        robot_position = next_pos

    # Horizontal movement is simple, just slight modification from the previous solution to account for wider boxes.
    elif grid[next_pos[0], next_pos[1]] in '[]' and direction in '<>':
        n_box = 1
        box_pos = next_pos
        
        while True:
            next_box = (box_pos[0] + 2*dr, box_pos[1] + 2*dc)
            if box_pos[0] + 2*dr >= len(grid) or box_pos[1] + 2*dc >= len(grid[0]):
                break
            if grid[next_box[0], next_box[1]] == '#':
                break
            elif grid[next_box[0], next_box[1]] in '[]':
                box_pos = next_box
                n_box += 1
                continue
            elif grid[next_box[0], next_box[1]] == '.':
                r, c = robot_position
                slice_range = slice(c, c+(2*n_box+1)+1) if direction=='>' else slice(c-(2*n_box+1), c+1)
                grid[r, slice_range] = np.roll(grid[r, slice_range], 1 if direction=='>' else -1)
                robot_position = (robot_position[0] + dr, robot_position[1] + dc)
            break

    # Vertical movement is different, as one box might push '2' boxes, and those two might push '3', etc.
    elif grid[next_pos[0], next_pos[1]] in '[]' and direction in '^v':
        n_box = 1
        box_pos_vec = [next_pos] # Vector of positions occupied by the boxes. (Boxes now occupy 2 cells.)

        if grid[next_pos[0], next_pos[1]] == '[': box_pos_vec.append((next_pos[0], next_pos[1] + 1))
        else: box_pos_vec.append((next_pos[0], next_pos[1] - 1))
        
        all_box_pos_vec = box_pos_vec.copy()

        while True:
            next_box_vec = sorted([(b_p[0] + dr, b_p[1] + dc) for b_p in box_pos_vec], key=lambda x: x[1])
            c1 = 0
            # Edge points
            if grid[next_box_vec[0][0], next_box_vec[0][1]] == ']':
                next_box_vec.append((next_box_vec[0][0], next_box_vec[0][1] - 1))
                c1+=1
            if grid[next_box_vec[-(1+c1)][0], next_box_vec[-(1+c1)][1]] == '[':
                next_box_vec.append((next_box_vec[-(1+c1)][0], next_box_vec[-(1+c1)][1] + 1))
            # Re-sort the list
            next_box_vec = sorted(next_box_vec, key=lambda x: x[1])

            # Remove leading and trailing '.'s from next_box_vec
            while next_box_vec and grid[next_box_vec[0][0], next_box_vec[0][1]] == '.':
                next_box_vec.pop(0)
                # If list becomes empty, break the loop.
                if not next_box_vec: break
            while next_box_vec and grid[next_box_vec[-1][0], next_box_vec[-1][1]] == '.':
                next_box_vec.pop(-1)
                if not next_box_vec: break

            # If any of the elements is '#', break.
            if any(grid[n_b[0], n_b[1]] == '#' for n_b in next_box_vec):
                break

            # If we find other boxes, push them as well.
            elif any(grid[n_b[0], n_b[1]] in '[]' for n_b in next_box_vec):
                box_pos_vec = next_box_vec
                n_box += 1
                all_box_pos_vec.extend(box_pos_vec)
                continue

            # If we find empty cells, move the robot and the boxes.
            else:
                sorted_box_pos_vec = sorted(all_box_pos_vec, key=lambda x: x[0]) if direction == '^' else \
                                     sorted(all_box_pos_vec, key=lambda x: x[0], reverse=True)
                for box_pos in sorted_box_pos_vec:
                    if grid[box_pos[0], box_pos[1]] == '[': 
                        grid[box_pos[0] + dr, box_pos[1] + dc] = '['; grid[box_pos[0], box_pos[1]] = '.'
                    elif grid[box_pos[0], box_pos[1]] == ']':
                        grid[box_pos[0] + dr, box_pos[1] + dc] = ']'; grid[box_pos[0], box_pos[1]] = '.'
                    else:
                        grid[box_pos[0] + dr, box_pos[1] + dc] = '.'; grid[box_pos[0], box_pos[1]] = '.'

                # Move the robot by 1 position.
                grid[robot_position[0] + dr, robot_position[1] + dc] = '@'; grid[robot_position[0], robot_position[1]] = '.'
                robot_position = (robot_position[0] + dr, robot_position[1] + dc)
            break
    else:
        pass

p2 = sum(100*i + j for i, row in enumerate(grid)
            for j, cell in enumerate(row) if cell == '[')
print(f'Part 2: {p2}')

Part 2: 1519991


This is really messy, but I'm too lazy to clean it up.